# QSAR Aquatic Toxicity

## Description

The QSAR Aquatic Toxicity dataset predicts the aquatic toxicity towards the Pimephales Promelas and Daphina Magna fish species givem certain molecular properties and an experimental response.

The column attributes are as follows:

| Column Name | Property |
| - | - |
| tpsa | TPSA(tot) |
| saacc | SAacc |
| h050 | H-050 |
| mlogp | MLOGP |
| rdchi | RDCHI |
| gats1p | GATS1p |
| nn | nN |
| c040 | C-040 |
| lc50 | quantitative response, LC50 [-LOG(mol/L)] |

[Source](http://archive.ics.uci.edu/ml/datasets/QSAR+aquatic+toxicity)

## Importing the Dataset

In [ ]:
import pandas as pd

column_names = ['tpsa',
                'saacc',
                'h050',
                'mlogp',
                'rdchi',
                'gats1p',
                'nn',
                'c040',
                'lc50']

with open("../../datasets/regression/qsar_aquatic_toxicity.csv", "r") as dataset_file:
    raw_data = pd.read_csv(dataset_file, delimiter=';', names=column_names)

## Preparing the Dataset

In [ ]:
# No preperations need to be done.
processed_data = raw_data

The following block prints the shape and column datatypes of the processed dataset.

In [ ]:
print(processed_data.shape)
print(processed_data.dtypes)